In [2]:
t
import numpy as np
import pandas as pd

# Convert all datasets to GML
These are just pieces of useful code, not a pipeline. You don't need to use it, as soon as all the graphs already prepared.

In [9]:
# variant 1: for .edges and .clusters
graph_name = 'karate'

G = nx.read_edgelist(f'./raw_graphs/{graph_name}/{graph_name}.edges')

clusters = {}
with open(f'./raw_graphs/{graph_name}/{graph_name}.clusters') as f:
    for line in f:
        name, cluster = line[:-1].split('\t')
        clusters[name] = cluster

nx.set_node_attributes(G, clusters, name='gt')
nx.write_gml(G, f'./gml_graphs/{graph_name}.gml')

In [7]:
# variant 2: for .edges and .nodes
graph_name = 'wisconsin'

df = pd.read_table(f'./raw_graphs/webkb/{graph_name}/webkb-{graph_name}.edges', header=2, names=['id', 'source', '|', 'target'])
edges = df[['source', 'target']]
G = nx.convert_matrix.from_pandas_edgelist(edges)

df = pd.read_table(f'./raw_graphs/webkb/{graph_name}/webkb-{graph_name}.nodes', header=1)
clusters = df[['cat=course,faculty,project,staff,student:label']]
clusters = dict(clusters.reset_index().to_numpy().tolist())


nx.set_node_attributes(G, clusters, name='gt')
nx.write_gml(G, f'./gml_graphs/{graph_name}.gml')

In [42]:
# variant 3: for newsgroups
graph_name = 'news_5cl_3'

A = []
with open(f'./raw_graphs/newsgroup/{graph_name}_Docr.csv') as f:
    for line in f:
        A.append(line[:-1].split(','))
G = nx.from_numpy_matrix(np.array(A, dtype=np.float32))

clusters = []
with open(f'./raw_graphs/newsgroup/{graph_name}_classeo.csv') as f:
    for line in f:
        clusters.append(line[:-1])
clusters = dict(enumerate(clusters))
nx.set_node_attributes(G, clusters, name='gt')
nx.write_gml(G, f'./gml_graphs/{graph_name}.gml')

In [10]:
# save connected graphs
for folder, path in paths:
    new_basepath = f'./gml_connected_subgraphs/{folder}' if folder != '' else f'./gml_connected_subgraphs'
    new_filename = path.split('/')[-1]
    print(new_filename)
    try:
        os.mkdir(new_basepath)
    except:
        pass
    G = nx.read_gml(path)
    if nx.is_directed(G):
        G = G.to_undirected()
    Gc = max([G.subgraph(c) for c in nx.connected_components(G)], key=len)
    nx.write_gml(Gc, f'{new_basepath}/{new_filename}')